# Retirement Planning Tool

This tool simulates a series of cashflows against historical market returns. 

This approach is a generalization of the approach behind the 4% rule. The 4% rule does not account for changes in your cashflow needs such as the start of Social Security, and assumes a 30 year timeframe. This tool allows you to choose your timeframe and models uneven cashflows. 

## Disclaimer

The information in this repo is provided "as is" for general informational purposes only. We make no warranties regarding its accuracy or completeness. We are not liable for any losses or damages resulting from your use of or reliance on this information. Use at your own risk.

In [1]:
import numpy as np
import xlrd

def load_data():
    book = book = xlrd.open_workbook("ie_data.xls")
    sheet = book.sheet_by_name("Data")
    dates = sheet.col_values(0) # Date
    prices = sheet.col_values(9) # Real Total Return Price
    assert len(dates) == 1853, len(dates)
    assert len(prices) == 1853, len(dates)
    return dates, prices

dates, prices = load_data()


In [2]:
def etl(dates, prices):
    """Cleanse the data: Trim the headers and excess white space at the end."""

    print(dates[:10])
    print(dates[-10:])

    print(prices[:10])
    print(prices[-10:])

    dates = dates[8:-1]
    prices = prices[8:-1]

    print(dates[0], dates[-10:])
    print(prices[0], prices[-10:])


    N = len(dates)

    print("Data consists of", N, "entries from", dates[0], "to", dates[-1])

    for idx, p in enumerate(prices):
        try: 
            float(p)
        except ValueError:
            print("Error: ", idx, p)


    dates = np.array(dates, dtype=np.float32)
    prices = np.array(prices, dtype=np.float32)

    return dates, prices

dates, prices = etl(dates, prices)

['', 'Stock Market Data Used in "Irrational Exuberance" Princeton University Press, 2000, 2005, 2015, updated', 'Robert J. Shiller ', '', '', '', '', 'Date', 1871.01, 1871.02]
[2023.12, 2024.01, 2024.02, 2024.03, 2024.04, 2024.05, 2024.06, 2024.07, 2024.08, '']
['', '', '', '', 'Real', 'Total', 'Return', 'Price', 111.94505242623511, 110.6262185217641]
[3172216.258905917, 3246908.2759480295, 3362452.0564336954, 3450516.754294489, 3402471.1361407707, 3482336.4999193936, 3604784.4188651205, 3685951.498636391, 3624862.5632474283, '']
1871.01 [2023.11, 2023.12, 2024.01, 2024.02, 2024.03, 2024.04, 2024.05, 2024.06, 2024.07, 2024.08]
111.94505242623511 [3013110.7220989885, 3172216.258905917, 3246908.2759480295, 3362452.0564336954, 3450516.754294489, 3402471.1361407707, 3482336.4999193936, 3604784.4188651205, 3685951.498636391, 3624862.5632474283]
Data consists of 1844 entries from 1871.01 to 2024.08


In [4]:
def calculate_equity_returns(prices):
    returns = [None] * (len(prices) - 1)
    for i in range (1, len(prices)):
        returns[i-1] = (prices[i] - prices[i-1]) / prices[i-1]
    return returns

equity_returns = calculate_equity_returns(prices)
print(np.mean(equity_returns))
print(np.std(equity_returns))

0.0064776065
0.040820498


In [26]:
def calculcate_portfolio_returns(equity_returns, annual_tips_returns=0.02, equity_ratio=0.75, tips_ratio=0.25, tax_rate = 0.25):
    """Blend equities and bonds returns for a portfolio. Assumes regular rebalancing.
    
    Args:
        equity_returns: list of equity returns
        annual_tips_returns: annual TIPS returns
        equity_ratio: ratio of equities in the portfolio
        tips_ratio: ratio of TIPS in the portfolio
        tax_rate: blended tax rate assuming some taxable div & cap gains, and some in a tax protected IRA.
    """
    portfolio_returns = [None] * len(equity_returns)
    for i in range(len(equity_returns)):
        pretax_return = equity_ratio * equity_returns[i] + tips_ratio * annual_tips_returns / 12.0
        portfolio_returns[i] = (1 - tax_rate) * pretax_return
    return portfolio_returns

portfolio_returns = calculcate_portfolio_returns(equity_returns)

In [30]:
def simulate(cashflows, dates, returns):
    """Simulate the value of a portfolio given historical prices and cashflows.

    Args:
        cashflows: list of cashflows. The zero index should be positive to represent your retirement funds.
        historical_prices: list of historical equity prices from Schiller. 
    """
        
    C = len(cashflows)
    R = len(returns)

    assert C > 1, "Need at least two cashflows"

    num_sims = R - C + 1

    assert num_sims > 0, f"Cashflows must be less than returns. C={C}, R={R}"

    wins = 0
    losses = 0
    total = 0
    bad_retirement_years = []
    months_before_bankrupt = []
    for i in range(num_sims):
        portfolio = cashflows[0]
        assert portfolio > 0, "Initial portfolio must be positive"

        for c in range(1, C):
            portfolio += cashflows[c] # Spend that money.
            portfolio *= (1 + returns[i+c-1]) # Get a return on that money.
            if portfolio < 0:
                losses += 1
                total +=1
                bad_retirement_years.append(dates[i])
                months_before_bankrupt.append(c)
                break
        else: # no break
            wins += 1
            total += 1

    return wins, losses, total, bad_retirement_years, months_before_bankrupt

def test_simulate():
    """Test against most of the parameters of the original 4% rule study. 

    Note that we do not find that a retiree would run out of money if retiring in 1969
    despite a terrible sequence of returns in the 1970s becuase the US now offers
    TIPS, which would not have lost money. A regulatory / governance risk for your
    retirement planning is if the US Govt eliminiates TIPS, or more likely, fudges
    the calculation of inflation that TIPS are based on.

    We also calculate monthly returns instead of annual returns, since
    Shiller now provides monthly returns. 
    """

    # global: dates
    # global: portfolio_returns

    # Arrange
    dates_slice = dates[660:1465] # 1926 through 1992, like the original study
    returns_slice = portfolio_returns[660:1465] 
    savings = 100.0
    spend = -savings * 0.04 / 12 # Spend 4% annual or 0.33% monthly in real terms. 
    num_periods = 30 * 12 # The original paper modeled 50 years, finding there were always 30 years of cashflow. 
    cashflows = [savings] + [spend] * num_periods

    # Act
    wins, losses, total, bad_retirement_years, months_before_bankrupt = simulate(cashflows, dates_slice, returns_slice)

    # Assert
    assert wins + losses == total
    assert wins / total > 0.99, f"{wins} / {total} = {wins / total}"

    print("Test passed: success rate for 30 years of 4% rule is:", wins / total, "with", total, "trials.")
    print("The bad retirement years are:", bad_retirement_years)
    print("The months of retirement are:", months_before_bankrupt)


test_simulate()

Test passed: success rate for 30 years of 4% rule is: 0.9955056179775281 with 445 trials.
The bad retirement years are: [1929.08, 1929.09]
The months of retirement are: [350, 331]


In [40]:
def test_simulate2():
    """Test a 5% rule against all equity data for a 50 year retirement.

    Only 75% of the time will the retiree not run out of money.

    Note that a 3% rule would have a 100% success rate. 
    """

    # global: dates
    # global: portfolio_returns

    # Arrange
    dates_slice = dates
    returns_slice = portfolio_returns
    savings = 100.0
    spend = -savings * 0.04 / 12 # Spend 4% annual or 0.33% monthly in real terms. 
    num_periods = 50 * 12 # We want to retire at 55 and live to 105!
    cashflows = [savings] + [spend] * num_periods

    # Act
    wins, losses, total, bad_retirement_years, months_before_bankrupt = simulate(cashflows, dates_slice, returns_slice)

    # Assert
    assert wins + losses == total
    assert 0.7 < wins / total <= 1.0, f"{wins} / {total} = {wins / total}"

    print("Test passed: success rate for 50 years of 4% rule is:", wins / total, "with", total, "trials.")
    print("The bad retirement years are:", bad_retirement_years)
    print("The months of retirement are:", months_before_bankrupt)


test_simulate2()

Test passed: success rate for 50 years of 4% rule is: 0.7626709573612228 with 1243 trials.
The bad retirement years are: [1886.11, 1899.01, 1899.02, 1899.03, 1899.04, 1899.05, 1899.08, 1900.12, 1901.01, 1901.02, 1901.03, 1901.04, 1901.05, 1901.06, 1901.07, 1901.08, 1901.09, 1901.1, 1901.11, 1901.12, 1902.01, 1902.02, 1902.03, 1902.04, 1902.05, 1902.06, 1902.07, 1902.08, 1902.09, 1902.1, 1902.11, 1902.12, 1903.01, 1903.02, 1903.03, 1903.04, 1903.05, 1904.1, 1904.11, 1904.12, 1905.01, 1905.02, 1905.03, 1905.04, 1905.05, 1905.06, 1905.07, 1905.08, 1905.09, 1905.1, 1905.11, 1905.12, 1906.01, 1906.02, 1906.03, 1906.04, 1906.05, 1906.06, 1906.07, 1906.08, 1906.09, 1906.1, 1906.11, 1906.12, 1907.01, 1907.02, 1907.03, 1907.04, 1908.08, 1908.09, 1908.1, 1908.11, 1908.12, 1909.01, 1909.02, 1909.03, 1909.04, 1909.05, 1909.06, 1909.07, 1909.08, 1909.09, 1909.1, 1909.11, 1909.12, 1910.01, 1910.02, 1910.03, 1910.04, 1910.05, 1910.1, 1910.11, 1910.12, 1911.01, 1911.02, 1911.03, 1911.04, 1911.05, 1911